Import Data

In [ ]:
!pip install keras_tuner

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

import keras_tuner as kt
from matplotlib import pyplot
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from keras_tuner.applications import HyperResNet
import numpy as np
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data() 
assert x_test.shape == ( 10000 , 32 , 32 , 3)
assert x_train.shape == ( 50000 , 32 , 32 , 3)
assert y_test.shape == ( 10000 , 1)
assert y_train.shape == ( 50000 , 1)
class_count = 10 

Normalize Data

In [ ]:
x_train = np.expand_dims(x_train,-1).astype("float32")/255.0
x_test = np.expand_dims(x_test,-1).astype("float32")/255.0
y_train = keras.utils.to_categorical(y_train,class_count)
y_test = keras.utils.to_categorical(y_test,class_count)


print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3, 1)
(50000, 10)


Create Base Model

In [ ]:
def create_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))

	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

In [ ]:
model1 = create_model()
model1.fit(x_train,y_train,epochs = 2, validation_split = .2, batch_size = 100)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/2
400/400 [==============================] - 130s 323ms/step - loss: 1.8930 - accuracy: 0.3255 - val_loss: 1.6976 - val_accuracy: 0.4022
Epoch 2/2
400/400 [==============================] - 120s 301ms/step - loss: 1.6023 - accuracy: 0.4341 - val_loss: 1.5887 - val_accuracy: 0.4354


Epoch 1/2
400/400 [==============================] - 120s 300ms/step - loss: 1.8632 - accuracy: 0.3395 - val_loss: 1.6842 - val_accuracy: 0.3983

Epoch 2/2
400/400 [==============================] - 120s 300ms/step - loss: 1.5738 - accuracy: 0.4454 - val_loss: 1.5424 - val_accuracy: 0.4501

For my baseline model, it acheived a test accuracy of 45.01%
This is a good start, but to improve this score, we must add in more extra dense layers and convolution layers.

In [ ]:
def create_model2():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
 
 
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
 
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
 
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))

	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

In [ ]:
model2 = create_model2()
model2.fit(x_train,y_train,epochs = 3, validation_split = .2, batch_size = 50)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/3
800/800 [==============================] - 192s 240ms/step - loss: 1.8625 - accuracy: 0.3218 - val_loss: 1.6129 - val_accuracy: 0.4133
Epoch 2/3
800/800 [==============================] - 186s 233ms/step - loss: 1.5259 - accuracy: 0.4476 - val_loss: 1.5500 - val_accuracy: 0.4433
Epoch 3/3
800/800 [==============================] - 184s 230ms/step - loss: 1.3964 - accuracy: 0.4987 - val_loss: 1.3565 - val_accuracy: 0.5160


For my second model, I increased the amount of epochs and lowered the batch size.

Epoch 1/3
800/800 [==============================] - 181s 226ms/step - loss: 1.8501 - accuracy: 0.3307 - val_loss: 1.5716 - val_accuracy: 0.4267

Epoch 2/3
800/800 [==============================] - 186s 233ms/step - loss: 1.4856 - accuracy: 0.4601 - val_loss: 1.3915 - val_accuracy: 0.4964

Epoch 3/3
800/800 [==============================] - 187s 233ms/step - loss: 1.3532 - accuracy: 0.5178 - val_loss: 1.3534 - val_accuracy: 0.5127

Tuner

The tuner took a long time to run, and after I ran it I broke the code. However, I did learn that I need to increase the epochs significantly. 

In [ ]:
#tuner = kt.RandomSearch(
#    create_model,
#    objective="val_accuracy",
#    max_trials=2,
#    overwrite=True,
#    directory="my_dir",
#    project_name="MachineLearningProject2",
#)
#stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)

#tuner.search(x_train, y_train, epochs =4, validation_split = .2, callbacks =[stop_early])

#best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

#print("Layers: " + best_hps.get('num_layers'))
#print("Units_0: " + best_hps.get('units_0'))
#print("Units_1: " + best_hps.get('units_1'))
#print("Units_2: " + best_hps.get('units_2'))
#print("Lr: " + best_hps.get('lr'))
#print("dropout: " + best_hps.get('dropout'))
#print("Activation: " + best_hps.get('activation'))
#print("Epochs: " + best_hps.get('tuner/epochs'))


In [ ]:
def create_model3():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))

	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

In [ ]:
model3 = create_model3()
model3.fit(x_train,y_train,epochs = 10, validation_split = .2, batch_size = 50)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/10
800/800 [==============================] - 131s 163ms/step - loss: 1.7703 - accuracy: 0.3731 - val_loss: 1.5783 - val_accuracy: 0.4368
Epoch 2/10
800/800 [==============================] - 130s 162ms/step - loss: 1.4426 - accuracy: 0.4871 - val_loss: 1.4077 - val_accuracy: 0.5091
Epoch 3/10
800/800 [==============================] - 130s 162ms/step - loss: 1.2770 - accuracy: 0.5490 - val_loss: 1.2641 - val_accuracy: 0.5589
Epoch 4/10
800/800 [==============================] - 129s 161ms/step - loss: 1.1574 - accuracy: 0.5922 - val_loss: 1.1762 - val_accuracy: 0.5925
Epoch 5/10
800/800 [==============================] - 128s 161ms/step - loss: 1.0617 - accuracy: 0.6293 - val_loss: 1.1705 - val_accuracy: 0.5917
Epoch 6/10
800/800 [==============================] - 128s 161ms/step - loss: 0.9881 - accuracy: 0.6561 - val_loss: 1.0698 - val_accuracy: 0.6305
Epoch 7/10
800/800 [==============================] - 129s 161ms/step - loss: 0.9213 - accuracy: 0.6805 - val_loss: 1.0458 -

Epoch 1/10
800/800 [==============================] - 130s 161ms/step - loss: 1.7543 - accuracy: 0.3732 - val_loss: 1.5349 - val_accuracy: 0.4472

Epoch 2/10
800/800 [==============================] - 128s 160ms/step - loss: 1.3931 - accuracy: 0.5024 - val_loss: 1.3896 - val_accuracy: 0.5182

Epoch 3/10
800/800 [==============================] - 131s 164ms/step - loss: 1.2511 - accuracy: 0.5580 - val_loss: 1.2748 - val_accuracy: 0.5487

Epoch 4/10
800/800 [==============================] - 129s 161ms/step - loss: 1.1532 - accuracy: 0.5925 - val_loss: 1.1749 - val_accuracy: 0.5874

Epoch 5/10
800/800 [==============================] - 128s 160ms/step - loss: 1.0617 - accuracy: 0.6266 - val_loss: 1.1340 - val_accuracy: 0.6066

Epoch 6/10
800/800 [==============================] - 129s 161ms/step - loss: 0.9868 - accuracy: 0.6551 - val_loss: 1.0975 - val_accuracy: 0.6204

Epoch 7/10
800/800 [==============================] - 127s 158ms/step - loss: 0.9215 - accuracy: 0.6767 - val_loss: 1.0591 - val_accuracy: 0.6281

Epoch 8/10
800/800 [==============================] - 127s 158ms/step - loss: 0.8582 - accuracy: 0.7033 - val_loss: 1.0176 - val_accuracy: 0.6468

Epoch 9/10
800/800 [==============================] - 126s 158ms/step - loss: 0.8021 - accuracy: 0.7222 - val_loss: 1.0067 - val_accuracy: 0.6529

Epoch 10/10
800/800 [==============================] - 126s 157ms/step - loss: 0.7470 - accuracy: 0.7400 - val_loss: 0.9939 - val_accuracy: 0.6551

Model 3 has a significant improvement over the previous 2 models. It has an accuracy of 74% and a val_accuracy of 65.51%

Increasing the Epoch had a significant effect on the outcome.